In [12]:
import h5py
import numpy as np
import pandas as pd
import h5py
import numpy as np
from scipy.sparse import csr_matrix

# Open the file
with h5py.File('bmcite.h5Seurat', 'r') as f:
    # Explore the file structure and contents
    # This will show you the different groups and datasets within the HDF5 file
    def printname(name):
        print(name)
    f.visit(printname)

active.ident
active.ident/levels
active.ident/values
assays
assays/ADT
assays/ADT/counts
assays/ADT/counts/data
assays/ADT/counts/indices
assays/ADT/counts/indptr
assays/ADT/data
assays/ADT/features
assays/ADT/variable.features
assays/RNA
assays/RNA/counts
assays/RNA/counts/data
assays/RNA/counts/indices
assays/RNA/counts/indptr
assays/RNA/data
assays/RNA/data/data
assays/RNA/data/indices
assays/RNA/data/indptr
assays/RNA/features
assays/RNA/meta.features
assays/RNA/meta.features/_index
assays/RNA/meta.features/vst.mean
assays/RNA/meta.features/vst.variable
assays/RNA/meta.features/vst.variance
assays/RNA/meta.features/vst.variance.expected
assays/RNA/meta.features/vst.variance.standardized
assays/RNA/variable.features
cell.names
commands
commands/FindModalityWeights
commands/FindModalityWeights/cross.contant.list
commands/FindModalityWeights/cross.contant.list__names__
commands/FindModalityWeights/dims.list
commands/FindModalityWeights/dims.list__names__
commands/FindModalityWeights/k

In [226]:
with h5py.File('bmcite.h5Seurat', 'r') as file:
    # Replace 'dataset_name' with the actual name of the dataset you want to retrieve
    
    # Protein data (raw)
    data = file['assays/ADT/counts/data'][:]
    indices = file['assays/ADT/counts/indices'][:]
    indptr = file['assays/ADT/counts/indptr'][:]
    dim = (len(indptr) - 1, max(indices) + 1)
    adt_matrix = csr_matrix((data, indices, indptr))
    
    # Protein data (after normalization)
    adt_counts_group = file['assays/ADT/data']
    adt_counts_data = np.array(adt_counts_group)
    adt_counts_df = pd.DataFrame(adt_counts_data)
    
    # Protein features
    adt_feature = file['assays/ADT/features']
    adt_feature_data = np.array(adt_feature)
    adt_feature_df = pd.DataFrame(adt_feature_data)
    adt_feature_df = adt_feature_df.applymap(lambda x: x.decode('utf-8') if isinstance(x, bytes) else x)
    
    
    # RNA data (raw)
    # rna_counts_group = file['assays/RNA/counts']
    # rna_counts_data = np.array(rna_counts_group)
    # rna_counts_df = pd.DataFrame(rna_counts_data)
    data = file['assays/RNA/counts/data'][:]
    indices = file['assays/RNA/counts/indices'][:]
    indptr = file['assays/RNA/counts/indptr'][:]
    shape = (file['assays/RNA/counts/indptr'].shape[0] - 1, file['assays/RNA/meta.features/_index'].shape[0])
    gene_expression_matrix = csr_matrix((data, indices, indptr), shape=shape)
    
    # RNA Features
    rna_feature = file['assays/RNA/features']
    rna_feature_data = np.array(rna_feature)
    rna_feature_df = pd.DataFrame(rna_feature_data)
    rna_feature_df = rna_feature_df.applymap(lambda x: x.decode('utf-8') if isinstance(x, bytes) else x)
    
    # Cell name
    cell_names_feature = file['cell.names']
    cell_names_data = np.array(cell_names_feature)
    cell_names_df = pd.DataFrame(cell_names_data)
    cell_names_df = cell_names_df.applymap(lambda x: x.decode('utf-8') if isinstance(x, bytes) else x)
    
    cell_type1_feature = file['meta.data/celltype.l1']
    cell_type1_data = np.array(cell_type1_feature)
    cell_type1_df = pd.DataFrame(cell_type1_data)
    cell_type1_df = cell_type1_df.applymap(lambda x: x.decode('utf-8') if isinstance(x, bytes) else x)
    
    cell_type2_feature = file['meta.data/celltype.l2']
    cell_type2_data = np.array(cell_type2_feature)
    cell_type2_df = pd.DataFrame(cell_type2_data)
    cell_type2_df = cell_type2_df.applymap(lambda x: x.decode('utf-8') if isinstance(x, bytes) else x)
    
    cell_embeddings_feature = file['reductions/spca/cell.embeddings']
    cell_embeddings_data = np.array(cell_embeddings_feature)
    cell_embeddings_df = pd.DataFrame(cell_embeddings_data)
    cell_embeddings_df = cell_embeddings_df.applymap(lambda x: x.decode('utf-8') if isinstance(x, bytes) else x)

In [262]:
# Create anndata
import anndata
import pandas as pd
import numpy as np

# cell_names_df.set_index(cell_names_df[0], inplace=True)
# rna_feature_df.set_index(rna_feature_df[0], inplace=True)
# adt_feature_df.set_index(adt_feature_df[0], inplace=True)

# Assume X is your data matrix, obs is the metadata of the cells, and var is the information of the genes
# rna = anndata.AnnData(X=gene_expression_matrix, obs=cell_names_data, var=rna_feature_df)
# protein = anndata.AnnData(X=adt_matrix, obs=cell_names_data, var=adt_feature_df)

rna = anndata.AnnData(X=gene_expression_matrix)
protein = anndata.AnnData(X=adt_matrix)

In [263]:
rna.obs_names = np.array(cell_names_df[0])
protein.obs_names = np.array(cell_names_df[0])

In [264]:
adt_counts_df.set_index(protein.obs_names,inplace=True)
cell_embeddings_df.columns = rna.obs_names
adt_counts_df.columns =  protein.var_names
cell_type1_df.set_index(rna.obs_names,inplace=True)
cell_type2_df.set_index(rna.obs_names,inplace=True)

protein.obsm['normalized_counts'] = adt_counts_df
protein.obs['celltype_l1'] = cell_type1_df
protein.obs['celltype_l2'] = cell_type2_df
# protein.obsm['origin_embedding'] = cell_embeddings_df.T


rna.obs['celltype_l1'] = cell_type1_df
rna.obs['celltype_l2'] = cell_type2_df
# rna.obsm['origin_embedding'] = cell_embeddings_df.T

In [180]:
del(rna.obs[0])
del(rna.var[0])
del(protein.obs[0])
del(protein.var[0])

In [260]:
obsm_keys_as_str = {str(key): value for key, value in rna.obsm.items()}
rna.obsm = obsm_keys_as_str

In [265]:
rna.write_h5ad('BMNC_RNA.h5ad')
protein.write_h5ad('BMNC_ADT.h5ad')